In [87]:
%matplotlib inline
from collections import defaultdict, Counter
import glob
import os
import json

from IPython.core.display import HTML, Image
from matplotlib_venn import venn3
import pandas as pd
import pybedtools
import pysam
import gspread
from gscripts.general import parsers
from gscripts.general import dataviz
%load_ext autoreload
%autoreload 2
reload(parsers)
reload(pybedtools)

from oauth2client.client import SignedJwtAssertionCredentials

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [88]:
json_key = json.load(open("../public clip-588adbc137f3.json"))
scope = ['https://spreadsheets.google.com/feeds']

credentials = SignedJwtAssertionCredentials(json_key['client_email'], json_key['private_key'], scope)
gc = gspread.authorize(credentials)

sht1 = gc.open_by_url("https://docs.google.com/spreadsheets/d/1ZU2mQh54jentqvhR_oMnviLGWR8Nw_x338gULzKjNDI/edit#gid=0")
ws = sht1.worksheet("Sheet1")
list_of_lists = ws.get_all_values()
manifest = pd.DataFrame(list_of_lists[1:], columns=list_of_lists[0])
manifest.is_encode = manifest.is_encode == "TRUE"

In [89]:
def format_file(index, adapter):
    index_1 = illumina_adapters.ix[index.index_1].values[0]
    index_2 = illumina_adapters.ix[index.index_2].values[0]
    if index.Lane == "": #Incase we are doing a rapid run and there isn't lane info
        sample_name = "{}_{}-{}_{}.fastq.gz".format(index.Hiseq_file_name, index_2, index_1, adapter)
        dir_name = "Sample_{}".format(index.Hiseq_file_name)

    elif index.Lane.startswith("S"):
        name = index.Hiseq_file_name.split("-")[0].replace("_", "-")
        sample_name = "{}_{}_{}.fastq.gz".format(name, index.Lane, adapter)
        dir_name = index.Hiseq_file_name

    else:
        sample_name = "{}_{}-{}_{}_{}.fastq.gz".format(index.Hiseq_file_name, index_2, index_1, index.Lane, adapter)
        dir_name = "Sample_{}".format(index.Hiseq_file_name)
    return os.path.join(index.file_location, dir_name, sample_name)

In [90]:
input_manifest = pd.read_csv("../input_manifest.csv")
clip_manifest  = pd.read_csv("../clipseq_manifest.csv")

In [167]:
analysis_dir = "/home/gpratt/projects/encode/analysis/encode_v8/"
encode_v7 = parsers.clipseq_metrics(analysis_dir, iclip=True)

encode_v7["Fraction Collapsed"] = encode_v7['Usable Reads'] / encode_v7['Uniquely Mapped Reads'].astype(float)
encode_v7["Fraction Usable"] = encode_v7['Usable Reads'] / encode_v7['Input Reads'].astype(float)
unmerged_clip_manifest = encode_v7[["merged" not in index for index in encode_v7.index]]

In [168]:
filtered_encode_v7 = encode_v7[[ "Input Reads",
                                #'Trimmed bases',
                            "Reads Written",
                            "repetitive_count",
                            "Uniquely Mapped Reads",
                            "Uniquely mapped reads %",
                            "Usable Reads",
                            "Fraction Collapsed",
                            "Fraction Usable",
                            #"spot",
                            #"Num Peaks",
                            #"Passed QC"
                            ]]

In [169]:
HTML(filtered_encode_v7.to_html(formatters={"Input Reads" : parsers.commas,
                                     "Reads Written" : parsers.commas,
                                     "repetitive_count": parsers.commas,
                                     "Reads after Quality Filtering" : parsers.commas,
                                     "Uniquely Mapped Reads" : parsers.commas,
                                     "Usable Reads" : parsers.commas,
                                     "Num Peaks": parsers.commas
                                     } ))

,Input Reads,Reads Written,repetitive_count,Uniquely Mapped Reads,Uniquely mapped reads %,Usable Reads,Fraction Collapsed,Fraction Usable
202_02_PTBP1.merged,0,0,0,0,NaN,0,NaN,NaN
203-CLIPs_S6_R1.A01_203_02_HNRNPC,"2,859,437","2,735,512","5,298,570","1,702,445",78.82%,"1,574,516",0.924856,0.550638
203-CLIPs_S6_R1.B06_203_02_HNRNPC,"2,975,593","2,845,499","5,770,278","1,729,375",77.68%,"1,587,979",0.918239,0.533668
203-CLIPs_S6_R1.C01_203_01_HNRNPC,"2,183,329","2,073,163","3,680,603","1,435,632",83.95%,0,NaN,NaN
203-CLIPs_S6_R1.D08fixed_203_01_HNRNPC,"3,222,598","3,098,404","5,092,729","2,163,569",83.47%,"260,445",0.120377,0.080818
203-INPUT_S7_R1.unassigned,"16,965,536","16,776,482","35,357,905","2,581,714",50.76%,"2,516,566",0.974766,0.148334
203_02_HNRNPC.merged,0,0,0,0,NaN,0,NaN,NaN
205_02_IGF2BP1.merged,0,0,0,0,NaN,0,NaN,NaN
206-01_S8_R1.C01_206_01_HNRNPK,"4,228,741","3,784,137","2,946,818","2,335,968",74.73%,"2,247,791",0.962252,0.531551
206-01_S8_R1.D08fixed_206_01_HNRNPK,"3,173,611","2,967,005","2,158,625","1,886,623",75.38%,"1,814,884",0.961975,0.571867


In [225]:
manifest_df = pd.read_table("/home/gpratt/projects/encode/scripts/encode_v8.txt", header=None,
             names=['fastq', 'species', 'encode_id', 'barcodes', 'barcodes_len', 'more_barcodes'])
manifest_df['qc_name'] = manifest_df.fastq.apply(lambda x: ".".join(os.path.basename(x.split(";")[0]).split(".")[:2]))

tmp = pd.merge(manifest_df, unmerged_clip_manifest, left_on="qc_name", right_index=True)
final_qc_frame = pd.merge(manifest, tmp, left_on='qc_id', right_on='encode_id')

In [226]:
final_qc_frame['exp_id'] = final_qc_frame.ENCODE_ID.apply(lambda x:x.split("_")[0])
final_qc_frame['rep_num'] = final_qc_frame.ENCODE_ID.apply(lambda x:x.split("_")[1])

In [227]:
new_index = []
for index, row in final_qc_frame.iterrows():
    new_index.append([row.RBP, row.exp_id, row.rep_num, row.cell_type])
    
final_qc_frame.index = pd.MultiIndex.from_tuples(new_index)
final_qc_frame = final_qc_frame.sort_index()

In [228]:
filtered_final_qc_frame = final_qc_frame[[ "Input Reads",
                                #'Trimmed bases',
                            "Reads Written",
                            "repetitive_count",
                            "Uniquely Mapped Reads",
                            "Uniquely mapped reads %",
                            "Usable Reads",
                            "Fraction Collapsed",
                            "Fraction Usable",
                            "spot",
                            "Num Peaks",
                            "Passed QC"
                            ]]

In [231]:
HTML(filtered_final_qc_frame.to_html(formatters={"Input Reads" : parsers.commas,
                                     "Reads Written" : parsers.commas,
                                     "repetitive_count": parsers.commas,
                                     "Reads after Quality Filtering" : parsers.commas,
                                     "Uniquely Mapped Reads" : parsers.commas,
                                     "Usable Reads" : parsers.commas,
                                     "Num Peaks": parsers.commas
                                     } ))